### Identifying missing values in a tabular data

In [31]:
import pandas as pd
from io import StringIO

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

csv_data = \
'''
A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,
'''
df = pd.read_csv(StringIO(csv_data))
df

#to check the missing values in each columns
df.isnull().sum()

# You can always access the underlying numpy array
# by using the values attribute, As scikit-learn api
# sometimes accepts the numpy array as input
df.values

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


A    0
B    0
C    1
D    1
dtype: int64

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

### Eliminating training examples or features with missing values


In [12]:
# drop the rows with missing values
df.dropna(axis=0) #axis 0 for index, 1 for columns

# drop columns that have atlest one NaN in any row
df.dropna(axis=1)

# only drops rows where all columns are NaN
df.dropna(how='all')

# drop rows that have fewer than 4 real values
df.dropna(thresh=3)

# Only drop rows where NaN appears in specific columns
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0


,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### Imputing missing values

In [32]:
from sklearn.impute import SimpleImputer
import numpy as np

# mean imputation, replace missing values with mean of feature
# other parameter for strategu are median, most_frequent
# most_frequent useful for imputing categorical variable

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

# alternative method in pandas
df.fillna(df.mean())

# KNN Imputer based on k-nearest neighbors approach to missing values

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


### Handling Categorical data


In [33]:
# Ordincal features - they have order or can be sorted
# Nominal features - no order
import pandas as pd
df = pd.DataFrame([
    ['Red', 'M', 11.5, 'class1'],
    ['Blue', 'L', 13.5, 'class2'],
    ['Green', 'XL', 15.5, 'class1']
])
df.columns = ['color', 'size', 'price', 'classlabel']
df

# Mapping ordinal features manually
size_mapping = {
    'XL':3,
    'L':2,
    'M':1}

df['size'] = df['size'].map(size_mapping)
df

# define a inverse mapping dictionary
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

,color,size,price,classlabel
0,Red,M,11.5,class1
1,Blue,L,13.5,class2
2,Green,XL,15.5,class1


,color,size,price,classlabel
0,Red,1,11.5,class1
1,Blue,2,13.5,class2
2,Green,3,15.5,class1


0     M
1     L
2    XL
Name: size, dtype: object

### Encoding class labels



In [34]:
# the labels of the class in number.
# The class labels are not ordinal, it does not matter
# which integer number we asisgn to particular string
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

# mapping dictionary to transform the class labels
df['classlabel'] = df['classlabel'].map(class_mapping)
df

# inverse mapping, the reverse of key-value pairs
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df




{'class1': 0, 'class2': 1}

,color,size,price,classlabel
0,Red,1,11.5,0
1,Blue,2,13.5,1
2,Green,3,15.5,0


,color,size,price,classlabel
0,Red,1,11.5,class1
1,Blue,2,13.5,class2
2,Green,3,15.5,class1


In [36]:
from sklearn.preprocessing import LabelEncoder
# alternatively there is LabelEncoder implement in scikit-learn to achieve this
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

# can call inverse transfrom method to tranform the integer class labels
class_le.inverse_transform(y)

array([0, 1, 0])

array(['class1', 'class2', 'class1'], dtype=object)

### Performing one-hot encoding on nominal features

In [38]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

# Here we could make the msot common mistake
# in dealing with catgeorical data. 
# It is now assumed that green is larger than blues
# To use One-Hot Encoding. To create a new dummy feature for each unique 
# value in the nominal feature column

# blue can - blue=1, green=0, red=0
from sklearn.preprocessing import OneHotEncoder
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[2, 1, 11.5],
       [0, 2, 13.5],
       [1, 3, 15.5]], dtype=object)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [44]:
# If we want to selectively tranform columns in a multi-feature array
# Use of ColumnTranformer. accepts a list (name, tranformer, columns(s))
from sklearn.compose import ColumnTransformer

c_tranf = ColumnTransformer([
    ('ohehot', OneHotEncoder(), [0]),
    ('nothing', 'passthrough', [1, 2]) #passthrough 
])
c_tranf.fit_transform(X).astype(float)

array([[ 0. ,  0. ,  1. ,  1. , 11.5],
       [ 1. ,  0. ,  0. ,  2. , 13.5],
       [ 0. ,  1. ,  0. ,  3. , 15.5]])

In [46]:
pd.get_dummies(df[['price', 'color', 'size']])
# to avoid multicollinearity, as highly correlated matrix are 
# computationally difficult to invert

# to reduce correlation among variables, we remove one feature column
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

,price,size,color_Blue,color_Green,color_Red
0,11.5,1,False,False,True
1,13.5,2,True,False,False
2,15.5,3,False,True,False


,price,size,color_Green,color_Red
0,11.5,1,False,True
1,13.5,2,False,False
2,15.5,3,True,False


In [47]:
# to drop redudant column via OneHotEncoder
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([
    ('onehot', color_ohe, [0]),
    ('nothing', 'passthrough', [1, 2])
])
c_transf.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  1. , 11.5],
       [ 0. ,  0. ,  2. , 13.5],
       [ 1. ,  0. ,  3. , 15.5]])

### Partitioning a dataset into separate training and test datasets
